1. import 목록

In [42]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds
import nltk

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

2. tensorflow_datasets 목록
    + 본래 amazon_us_reviews를 다루려고 했으나 403 forbidden 메세지를 보아 더이상 tensorflow에서 지원하지 않는다는 것을 확인
    + 이에 따라 대체 데이터셋으로 yelp_polarity_reviews를 이용하기로 결정

In [43]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar100_n',
 'cifar10_1',
 'cifar10_corrupted',
 'cifar10_n',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'conll2002',
 'conll2003',
 'controlled_noisy_web_labels'

3. train 데이터

In [44]:
tfds_name = 'yelp_polarity_reviews'

train_dataset, train_info = tfds.load(name=tfds_name, split=tfds.Split.TRAIN, with_info=True)
test_dataset, test_info = tfds.load(name=tfds_name, split=tfds.Split.TEST, with_info=True)

4. 결측치 제거

In [45]:
train_dataset = train_dataset.filter(lambda data: data['text'] is not None)
test_dataset = test_dataset.filter(lambda data: data['text'] is not None)

In [46]:
print(train_dataset)
print(test_dataset)

<_FilterDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}>
<_FilterDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}>


In [47]:
print(train_info)
print(test_info)

tfds.core.DatasetInfo(
    name='yelp_polarity_reviews',
    full_name='yelp_polarity_reviews/0.2.0',
    description="""
    Large Yelp Review Dataset.
    This is a dataset for binary sentiment classification. We provide a set of 560,000 highly polar yelp reviews for training, and 38,000 for testing. 
    ORIGIN
    The Yelp reviews dataset consists of reviews from Yelp. It is extracted
    from the Yelp Dataset Challenge 2015 data. For more information, please
    refer to http://www.yelp.com/dataset
    
    The Yelp reviews polarity dataset is constructed by
    Xiang Zhang (xiang.zhang@nyu.edu) from the above dataset.
    It is first used as a text classification benchmark in the following paper:
    Xiang Zhang, Junbo Zhao, Yann LeCun. Character-level Convolutional Networks
    for Text Classification. Advances in Neural Information Processing Systems 28
    (NIPS 2015).
    
    
    DESCRIPTION
    
    The Yelp reviews polarity dataset is constructed by considering stars 1 an

4. seed(난수)를 생성한다

In [48]:
value = 42
tf.random.set_seed(value)
np.random.seed(value)

5. train_dataset을 list로 전환하여 train_x, train_y를 만든다.
    test_dataset에도 마찬가지로 적용한다.

In [49]:
# 열 정보 확인
column_names = list(train_info.features.keys())
print(column_names)

['text', 'label']


In [50]:
# train_x, train_y 만들기

datalist = list(train_dataset.as_numpy_iterator())

train_x = np.array([data['text'] for data in datalist])
train_y = np.array([data['label'] for data in datalist])

print(train_x[:5])
print(train_y[:5])

[b"The Groovy P. and I ventured to his old stomping grounds for lunch today.  The '5 and Diner' on 16th St and Colter left me with little to ask for.  Before coming here I had a preconceived notion that 5 & Diners were dirty and nasty. Not the case at all.\\n\\nWe walk in and let the waitress know we want to sit outside (since it's so nice and they had misters).  We get two different servers bringing us stuff (talk about service) and I ask the one waitress for recommendations.  I didn't listen to her, of course, and ordered the Southwestern Burger w/ coleslaw and started with a nice stack of rings.\\n\\nThe Onion Rings were perfectly cooked.  They looked like they were prepackaged, but they were very crispy and I could actually bite through the onion without pulling the entire thing out (don't you hate that?!!!)\\n\\nThe Southwestern Burger was order Medium Rare and was cooked accordingly.  Soft, juicy, and pink with a nice crispy browned outer layer that can only be achieved on a well

In [51]:
# test_x, test_y 만들기

datalist = list(test_dataset.as_numpy_iterator())

test_x = np.array([data['text'] for data in datalist])
test_y = np.array([data['label'] for data in datalist])

print(test_x[:5])
print(test_y[:5])

[b'Was not impressed, and will not return.'
 b'I went in to purchase overalls and was treated so rudely I had to walk out even though he had the item I wanted.  I will never step foot in this placec of business again.'
 b'This place really is horrible... Every time I wind up getting convinced to go here, I always walk out feeling like my pocket has been picked. The food isn\'t \\""bad,\\"" but at roughly $25-$30 per person, there are SO many better places to eat. To hell with Olive Garden.\\n\\nAnd this particular one has the worst-smelling parking lot in Las Vegas... every time I drive by (to go to B.J.\'s nearby, which is a thousand times better), it smells like a sewer main broke or something. \\n\\n2 stars only because the wait staff is friendly.'
 b'First time visit.....  enjoyed their little cheese biscuits .... Had the ribs,beef brisket,mashed taters with gravy, Mac and cheese was goooood. Very cheesy and creamy just how I like it and collard greens had a smoky taste to it, wond

6. 데이터 전처리
    + 모델에 적용시키기 위해서는 수치화된 데이터가 필요하다.
    + 하지만 text가 존재하기 때문에 Scikit-learn의 NLP 모델을 통해 tokenizer 및 lemmatizer을 진행한다.

In [74]:
# tokenizer - 문장을 단어로 token화시킨다.

nltk.download('punkt')

train_x = str(train_x)
test_x = str(test_x)

train_x_tokens = word_tokenize(train_x)
test_x_tokens = word_tokenize(test_x)
# print(train_tokens[:5])
# print(test_tokens[:5])

# 문장부호는 불필요하기 때문에 제거한다. 그리고 b라는 불필요한 단어가 출력되어 제거하였다.
train_x_words = [token for token in train_x_tokens if token.isalpha() and token != 'b']
test_x_words = [token for token in test_x_tokens if token.isalpha() and token != 'b']

print(train_x_words[:5])
print(test_x_words[:5])

['The', 'Groovy', 'and', 'I', 'ventured']
['not', 'impressed', 'and', 'will', 'not']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [75]:
# lemmatizer

nltk.download('wordnet')

train_x_lemmatize = [WordNetLemmatizer().lemmatize(word) for word in train_x_words]
test_x_lemmatize = [WordNetLemmatizer().lemmatize(word) for word in test_x_words]

print(train_x_lemmatize[:5])
print(test_x_lemmatize[:5])

['The', 'Groovy', 'and', 'I', 'ventured']
['not', 'impressed', 'and', 'will', 'not']


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


7. tokenizer 이후 lemmatizer을 진행하였으나 특정 단어에는 적용되지 않는 것을 확인하였다. (예: ventured) <br>
 => spacy를 사용하여 더욱 정교한 lemmatizer을 진행하였다.

In [77]:
    import spacy

    # 콘솔에 python -m spacy download en_core_web_sm를 입력하여 install을 해야 한다.
    nlp = spacy.load('en_core_web_sm')

    train_x_text = " ".join(train_x_words)
    test_x_text = " ".join(test_x_words)

    train_x_doc = nlp(train_x_text)
    test_x_doc = nlp(test_x_text)
    
    # 품사 태깅을 이용하여 동사의 형태만 lemmatizer 진행
    train_x_spacy = [token.lemma_ if token.pos_ == 'VERB' else token.text for token in train_x_doc]
    test_x_spacy = [token.lemma_ if token.pos_ == 'VERB' else token.text for token in test_x_doc]

    print(train_x_spacy[:5])
    print(test_x_spacy[:5])

['The', 'Groovy', 'and', 'I', 'venture']
['not', 'impressed', 'and', 'will', 'not']


8. impressed와 같이 여전히 lemmatizer가 적용되지 않은 사레들이 존재하지만 venture의 경우에는 정상적으로 lemmatizer가 진행되었다. <br>
 이제 본격적으로 모델을 작성하여 compile 후 fit을 적용시키기로 하였다.

9. 모델을 제작하기 전에 출력 데이터를 0과 1로 이진변환한다. (감성분석)

10. 모델 작성 - lstm 이용

In [ ]:
# # model 작성

# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Embedding(30000, 128))
# model.add(tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# model.summary()

11. 모델 compile

In [114]:
# # model compile

# opt = tf.keras.optimizers.Adam(learning_rate=0.01)
# model.compile(optimizer=opt, loss = 'binary_crossentropy', metrics=['accuracy'])

12. 모델 fit

In [ ]:
# model fit